In [ ]:
from clearml import Model, Logger
import pandas as pd
from keras.models import load_model
import numpy as np
from utility import preprocess_image, OptimizedRounder, mapping
from clearml import Task, TaskTypes
from PIL import Image
import os
import shap
from lime import lime_image
import matplotlib.pyplot as plt
from keras.utils import img_to_array,load_img
from skimage.segmentation import mark_boundaries
from keras.applications.efficientnet_v2 import preprocess_input

In [2]:
#model_path = Model(model_id="48a76b1277154398991d1d079db968ae").get_local_copy()
model = load_model('/home/abhijitbarman/abhijit/WS/DR/models_downloaded/beeef706a848dd90b41ebd61f8cd356c.final_model.h5')
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-s (Functiona  (None, 7, 7, 1280)       20331360  
 l)                                                              
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 1)                 1281      
                                                                 
Total params: 20,332,641
Trainable params: 20,178,769
Non-trainable params: 153,872
_________________________________________________________________
None


In [ ]:
# validation set
# Set N to the number of rows in val_df
N = val_df.shape[0]
# Create an empty numpy array of dimensions (N, im_size, im_size, 3) to hold the preprocessed validation images
x_val = np.empty((N, im_size, im_size, 3), dtype=np.uint8)
# Loop over each image ID in the id_code column of the val_df dataframe
for i, image_id in enumerate(tqdm_notebook(val_df['id_code'])):
    # Call the preprocess_image function to preprocess the image with the desired size im_size
    preprocessed_image = preprocess_image(f'{image_id}', desired_size=im_size)
    # Add the preprocessed image to the x_val numpy array
    x_val[i, :, :, :] = preprocessed_image

In [ ]:
data = pd.read_csv('jarvis_data/names.txt')
print(data.shape)
data['diagnosis'] = data.diagnosis.astype(float)
y_true = data.diagnosis.values.tolist()
ids = data.id_code.values.tolist()

In [ ]:
data

In [ ]:
data.diagnosis.value_counts()

In [ ]:
#LOAD & PREPARE DATA
#y_true = data.diagnosis.values.tolist()
x_val = np.empty((data.shape[0], 224, 224, 3), dtype=np.uint8)
counter = 0
multiplier = 0
incr = 1
for i,file in enumerate(data.id_code.values): 
    if i ==0:
        multiplier = 5 
    if i !=0 and i % multiplier ==0:
        counter += 1 
        incr += 1
        multiplier = multiplier + 5 * incr 
    file_path = 'jarvis_data/Train/'+ mapping.get(counter)+'/'+file
    img = load_img(file_path, target_size=(224, 224))
    img_array = img_to_array(img)
    x_val[i,:,:,:] = preprocess_input(img_array)

In [ ]:
y_pred = model.predict(x_val)
print(f'Model Prediction: {y_pred}')

In [ ]:
sum(y_pred > 1)

In [ ]:
optR = OptimizedRounder()
#optR.fit(y_pred, y_true)
coefficients = coefficients = [0.49964604, 1.55479703, 2.4369177,  3.26701671] # Learnt while training the efficient net model.
print(f'Coefficients: {coefficients}')
y_val_pred = optR.predict(y_pred, coefficients)
y_val_pred  = np.squeeze(y_val_pred)

In [ ]:
y_val_pred

In [ ]:
df = pd.DataFrame({'Actual':y_true,'Predicted':y_val_pred, 'id_code':ids})
print(df)
df.to_csv('demo_inference.csv', index=False)

In [ ]:
results = pd.read_csv('demo_inference.csv')
results.shape

In [ ]:
results.Actual.value_counts()

In [ ]:
results.head(17)

In [ ]:
correct_preds = results[results['Actual']==results['Predicted']]
correct_preds.shape

In [ ]:
correct_preds

In [ ]:
correct_preds.to_csv('correct_preds.csv', index=False)

In [ ]:
X = correct_preds.groupby('Actual').head(2)
X.shape

In [ ]:
X.head()

In [ ]:
x_val = np.empty((X.shape[0], 224, 224, 3), dtype=np.uint8)
for i,code,y in zip(list(range(X.shape[0])),X.id_code.values, X.Actual.values):
    file_path = 'jarvis_data/Train/'+ mapping.get(int(y)) +'/'+code
    #x_val[i,:,:,:] = preprocess_image(file_path)

    img = load_img(file_path, target_size=(224, 224))
    img_array = img_to_array(img)
    x_val[i,:,:,:] = preprocess_input(img_array)


In [ ]:
X.to_csv('X.csv',index=False)

In [ ]:
shap.maskers.Image

In [ ]:
masker = shap.maskers.Image('inpaint_telea',x_val[0].shape)
explainer = shap.Explainer(model, masker)
# here we use 500 evaluations of the underlying model to estimate the SHAP values
shap_values = explainer(x_val, max_evals=500,)
#shap.image_plot(shap_values)

In [ ]:
for svalue in range(shap_values.shape[0]):
    print(svalue)
    shap.image_plot(shap_values[svalue])

In [ ]:
#DEEP EXPLAINER
masker = shap.maskers.Image('inpaint_telea',x_val[0].shape)
explainer = shap.DeepExplainer(model, x_val)

In [ ]:

# here we use 500 evaluations of the underlying model to estimate the SHAP values
shap_values = explainer.shap_values(x_val)
#shap.image_plot(shap_values)

for svalue in range(shap_values.shape[0]):
    print(svalue)
    shap.image_plot(shap_values[svalue])

In [ ]:
#LIME 

# Create a LIME explainer
explainer = lime_image.LimeImageExplainer()
for i in range(x_val.shape[0]):
    # Explain the predictions made by the model
    explanation = explainer.explain_instance(x_val[i], model.predict, top_labels=1, hide_color=0, num_samples=1000)
    temp, mask = explanation.get_image_and_mask(0, positive_only=False, num_features=10, hide_rest=False)
    fig, axe = plt.subplots(figsize=(7, 3.5))
    plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))
    #if y_true[i]:
    #   axe.text(180, 180, 'Actual:'+y_true[i], bbox=dict(facecolor='red', alpha=0.0))
    #axe.text(180, 190, 'Predicted:'+y_val_pred[i], bbox=dict(facecolor='red', alpha=0.0))
    plt.show()
    fig.savefig('out_images/lime/'+str(i)+'.png')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define the two folders where your images are located
folder1 = "out_images/shap"
folder2 = "out_images/lime"

# Load the images from the two folders
images1 = [plt.imread(f"{folder1}/{i}.png") for i in range(0, 10)]
images2 = [plt.imread(f"{folder2}/{i}.png") for i in range(0, 10)]

dignosis = [int(x) for x in X.Actual.values]
dignosis_text = [mapping.get(i) for i in dignosis]

preds = []
for txt in dignosis_text:
    array_created = np.full((20, 20, 3),255, dtype = np.uint8)
    preds.append(array_created)

#images3 = [plt.imread(f"{folder2}/{i}.png") for i in range(0, 10)]

    
#dignosis_text


# Create a figure with two columns and ten rows
fig, axes = plt.subplots(nrows=10, ncols=3, figsize=(15, 30),gridspec_kw={"width_ratios": [1, 1,1]})

# Set the titles for each column
axes[0][0].set_title("Actual          SHAP")
axes[0][1].set_title("LIME")
axes[0][2].set_title("ACTUAL/PREDICTED")

# Loop through each row and plot the corresponding images from each folder
for i in range(10):
    axes[i][0].imshow(images1[i])
    axes[i][1].imshow(images2[i])
    axes[i][2].imshow(preds[i])
    axes[i][2].text(10,10,dignosis_text[i],bbox=dict(facecolor='Green', alpha=0.5))
    axes[i][2].text(10,15,dignosis_text[i],bbox=dict(facecolor='Orange', alpha=0.5))
    
    #axe.text(25, 25, 'Predicted:'+mapping.get(int(prediction[0].item())), bbox=dict(facecolor='red', alpha=0.0))

# Hide the x and y axis ticks for all subplots
for ax in axes.flatten():
    ax.set_xticks([])
    ax.set_yticks([])

# Adjust the spacing between the subplots
#plt.subplots_adjust(hspace=-0.6)

# Display the plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define the two folders where your images are located
folder1 = "out_images/shap"
folder2 = "out_images/lime"

# Load the images from the two folders
images1 = [plt.imread(f"{folder1}/{i}.png") for i in range(0, 10)]
images2 = [plt.imread(f"{folder2}/{i}.png") for i in range(0, 10)]

dignosis = [int(x) for x in X.Actual.values]
dignosis_text = [mapping.get(i) for i in dignosis]

preds = []
for txt in dignosis_text:
    array_created = np.full((20, 20, 3),255, dtype = np.uint8)
    preds.append(array_created)

#images3 = [plt.imread(f"{folder2}/{i}.png") for i in range(0, 10)]

    
#dignosis_text


# Create a figure with two columns and ten rows
fig, axes = plt.subplots(nrows=10, ncols=2, figsize=(8,30),gridspec_kw={"width_ratios": [1, 1]})

# Set the titles for each column
axes[0][0].set_title("Actual          SHAP")
#axes[0][1].set_title("LIME")
axes[0][1].set_title("ACTUAL/PREDICTED")

# Loop through each row and plot the corresponding images from each folder
for i in range(10):
    axes[i][0].imshow(images1[i])
    #axes[i][1].imshow(images2[i])
    axes[i][1].imshow(preds[i])
    axes[i][1].text(7,5,dignosis_text[i],bbox=dict(facecolor='Green', alpha=0.5))
    axes[i][1].text(7,15,dignosis_text[i],bbox=dict(facecolor='Orange', alpha=0.5))
    
    #axe.text(25, 25, 'Predicted:'+mapping.get(int(prediction[0].item())), bbox=dict(facecolor='red', alpha=0.0))

# Hide the x and y axis ticks for all subplots
for ax in axes.flatten():
    ax.set_xticks([])
    ax.set_yticks([])

# Adjust the spacing between the subplots
#plt.subplots_adjust(hspace=-0.6)

# Display the plot
plt.show()

In [ ]:
#SHAP 2
# select a set of background examples to take an expectation over
import shap
background = x_val


shap.explainers._deep.deep_tf.op_handlers["AddV2"] = shap.explainers._deep.deep_tf.passthrough #this solves the "shap_ADDV2" problem but another one will appear
shap.explainers._deep.deep_tf.op_handlers["FusedBatchNormV3"] = shap.explainers._deep.deep_tf.passthrough #this solves the next problem which allows you to run the DeepExplainer.
shap.explainers._deep.deep_tf.op_handlers["DepthwiseConv2dNative"] = shap.explainers._deep.deep_tf.passthrough #this solves the next problem which allows you to run the DeepExplainer.
#e = shap.DeepExplainer(model, images)

# explain predictions of the model on three images
e = shap.DeepExplainer(model, background)


In [ ]:
# ...or pass tensors directly
# e = shap.DeepExplainer((model.layers[0].input, model.layers[-1].output), background)
shap_values = e.shap_values(x_val[0:2],check_additivity=False)